In [1]:
from keras.models import Model,load_model
from keras.layers import Embedding
from keras.layers import Bidirectional
from keras.layers import LSTM
from keras.layers import Input
from keras.layers import Conv1D
from keras.layers import MaxPool1D
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers import Concatenate
from keras.layers import Dense, Activation
from keras.optimizers import RMSprop
from sklearn.preprocessing import LabelEncoder
import pandas as pd
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence

Using TensorFlow backend.


In [131]:
da = pd.read_csv('../amazon_reviews_us_Mobile_Electronics_v1_00.tsv', sep='\t', error_bad_lines=False)

b'Skipping line 35246: expected 15 fields, saw 22\n'
b'Skipping line 87073: expected 15 fields, saw 22\n'


In [140]:
import pandas as pd
da = pd.read_csv('../amazon_reviews_us_Mobile_Electronics_v1_00.tsv', sep='\t', error_bad_lines=False)
da = da[['review_body', 'verified_purchase']]
da = da.dropna()
X = da.review_body
Y = da.verified_purchase
le = LabelEncoder()
Y = le.fit_transform(Y)
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.15)
max_words = 1000
max_len = 20
tok = Tokenizer(num_words=max_words)
tok.fit_on_texts(X_train)
sequences = tok.texts_to_sequences(X_train)
sequences_matrix = sequence.pad_sequences(sequences,maxlen=max_len)
sequenceInput = Input(shape=(20,),dtype='int32')
embeddingLayer = Embedding(max_words,100,input_length=20)
embeddedSequences = embeddingLayer(sequenceInput)
gru1 = LSTM(128, return_sequences=False)(embeddedSequences)
out = Dense((1),activation='softmax')(gru1)
mod = Model(sequenceInput,[out])
mod.summary()
mod.compile(loss='binary_crossentropy',optimizer=RMSprop(),metrics=['accuracy'])
mod.fit(sequences_matrix,Y_train,batch_size=128,epochs=1,validation_split=0.2)

b'Skipping line 35246: expected 15 fields, saw 22\n'
b'Skipping line 87073: expected 15 fields, saw 22\n'


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_29 (InputLayer)        (None, 20)                0         
_________________________________________________________________
embedding_28 (Embedding)     (None, 20, 100)           100000    
_________________________________________________________________
lstm_39 (LSTM)               (None, 128)               117248    
_________________________________________________________________
dense_21 (Dense)             (None, 1)                 129       
Total params: 217,377
Trainable params: 217,377
Non-trainable params: 0
_________________________________________________________________
Train on 71296 samples, validate on 17825 samples
Epoch 1/1


KeyboardInterrupt: 

In [176]:
import json
with open('mod.json', 'w') as outfile:
    json.dump(json.loads(mod.to_json()), outfile)

In [175]:
model_from_json(mod.to_json())

In [186]:
j_data

{'backend': 'tensorflow',
 'class_name': 'Model',
 'config': {'input_layers': [['input_29', 0, 0]],
  'layers': [{'class_name': 'InputLayer',
    'config': {'batch_input_shape': [None, 20],
     'dtype': 'int32',
     'name': 'input_29',
     'sparse': False},
    'inbound_nodes': [],
    'name': 'input_29'},
   {'class_name': 'Embedding',
    'config': {'activity_regularizer': None,
     'batch_input_shape': [None, 20],
     'dtype': 'float32',
     'embeddings_constraint': None,
     'embeddings_initializer': {'class_name': 'RandomUniform',
      'config': {'maxval': 0.05, 'minval': -0.05, 'seed': None}},
     'embeddings_regularizer': None,
     'input_dim': 1000,
     'input_length': 20,
     'mask_zero': False,
     'name': 'embedding_28',
     'output_dim': 100,
     'trainable': True},
    'inbound_nodes': [[['input_29', 0, 0, {}]]],
    'name': 'embedding_28'},
   {'class_name': 'LSTM',
    'config': {'activation': 'tanh',
     'activity_regularizer': None,
     'bias_constrain

In [180]:
j_data = json.loads(data)

In [181]:
j_data.keys()

dict_keys(['class_name', 'backend', 'config', 'keras_version'])

In [183]:
j_data['config'].keys()

dict_keys(['input_layers', 'name', 'output_layers', 'layers'])

In [184]:
j_data['config']['layers']

[{'class_name': 'InputLayer',
  'config': {'batch_input_shape': [None, 20],
   'dtype': 'int32',
   'name': 'input_29',
   'sparse': False},
  'inbound_nodes': [],
  'name': 'input_29'},
 {'class_name': 'Embedding',
  'config': {'activity_regularizer': None,
   'batch_input_shape': [None, 20],
   'dtype': 'float32',
   'embeddings_constraint': None,
   'embeddings_initializer': {'class_name': 'RandomUniform',
    'config': {'maxval': 0.05, 'minval': -0.05, 'seed': None}},
   'embeddings_regularizer': None,
   'input_dim': 1000,
   'input_length': 20,
   'mask_zero': False,
   'name': 'embedding_28',
   'output_dim': 100,
   'trainable': True},
  'inbound_nodes': [[['input_29', 0, 0, {}]]],
  'name': 'embedding_28'},
 {'class_name': 'LSTM',
  'config': {'activation': 'tanh',
   'activity_regularizer': None,
   'bias_constraint': None,
   'bias_initializer': {'class_name': 'Zeros', 'config': {}},
   'bias_regularizer': None,
   'dropout': 0.0,
   'go_backwards': False,
   'implementation

In [185]:
j_a = json.loads(a)

In [119]:
sequenceInput = Input(shape=(20,),dtype='int32')
embeddingLayer = Embedding(max_words,100,input_length=20)
embeddedSequences = embeddingLayer(sequenceInput)
gru1 = LSTM(128, return_sequences=True)(embeddedSequences)
gru = Bidirectional(LSTM(128,return_sequences=True))(gru1)
filterSizes = [2,4,5]
filters = 100
convs=[]
avgs =[]
for fsize in filterSizes:
    conv = Conv1D(filters=filters,kernel_size=fsize,activation='relu')(gru)
    pool = MaxPool1D(pool_size=2)(conv)
    pool = Dropout(0.2)(pool)
    pool = Flatten()(pool)
    convs.append(pool)
convout = Concatenate(axis=-1)(convs)
out = Dense((1),activation='softmax')(convout)
mod = Model(sequenceInput,[out])
mod.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_24 (InputLayer)           (None, 20)           0                                            
__________________________________________________________________________________________________
embedding_23 (Embedding)        (None, 20, 100)      100000      input_24[0][0]                   
__________________________________________________________________________________________________
lstm_33 (LSTM)                  (None, 20, 128)      117248      embedding_23[0][0]               
__________________________________________________________________________________________________
bidirectional_23 (Bidirectional (None, 20, 256)      263168      lstm_33[0][0]                    
__________________________________________________________________________________________________
conv1d_41 

Train on 71296 samples, validate on 17825 samples
Epoch 1/1


KeyboardInterrupt: 

In [69]:
mod.to_json(indent=4)

{
    "class_name": "Model",
    "backend": "tensorflow",
    "config": {
        "name": "model_10",
        "input_layers": [
            [
                "input_10",
                0,
                0
            ]
        ],
        "layers": [
            {
                "name": "input_10",
                "class_name": "InputLayer",
                "config": {
                    "dtype": "int32",
                    "batch_input_shape": [
                        null,
                        20
                    ],
                    "name": "input_10",
                    "sparse": false
                },
                "inbound_nodes": []
            },
            {
                "name": "embedding_10",
                "class_name": "Embedding",
                "config": {
                    "embeddings_regularizer": null,
                    "name": "embedding_10",
                    "input_dim": 1000,
                    "activity_regularizer": null,
         

In [138]:
from keras.models import model_from_json

In [94]:
import json

In [129]:
a = json.loads(json_string)

In [100]:
a['config']

{'input_layers': [['input_12', 0, 0]],
 'layers': [{'class_name': 'InputLayer',
   'config': {'batch_input_shape': [None, 20],
    'dtype': 'int32',
    'name': 'input_12',
    'sparse': False},
   'inbound_nodes': [],
   'name': 'input_12'},
  {'class_name': 'Embedding',
   'config': {'activity_regularizer': None,
    'batch_input_shape': [None, 20],
    'dtype': 'float32',
    'embeddings_constraint': None,
    'embeddings_initializer': {'class_name': 'RandomUniform',
     'config': {'maxval': 0.05, 'minval': -0.05, 'seed': None}},
    'embeddings_regularizer': None,
    'input_dim': 1000,
    'input_length': 20,
    'mask_zero': False,
    'name': 'embedding_12',
    'output_dim': 128,
    'trainable': True},
   'inbound_nodes': [[['input_12', 0, 0, {}]]],
   'name': 'embedding_12'},
  {'class_name': 'LSTM',
   'config': {'activation': 'tanh',
    'activity_regularizer': None,
    'bias_constraint': None,
    'bias_initializer': {'class_name': 'Zeros', 'config': {}},
    'bias_regul